In [1]:
import numpy as np
import pandas as pd
import sqlite3
from pathlib import Path
from datetime import datetime
import backtrader as bt # 导入 Backtrader
import backtrader.indicators as btind # 导入策略分析模块
import backtrader.feeds as btfeeds # 导入数据模块
import  gc

# 快速连接数据库
class fast_db_conn:
    def __init__(self,db_path):
        self._db = db_path
    def __enter__(self):
        self._conn = sqlite3.connect(self._db)
        print("打开连接 {}".format(self._db))
        return self._conn
    def __exit__(self, exc_type, exc_val, exc_tb):
        self._conn.commit()
        self._conn.close()
        print("关闭连接 {}".format(self._db))

# 声明路径
cur_path = Path(r'.')
print("当前路径为 {}".format(cur_path.resolve()))
gitee_path = cur_path / Path(r'./../../..')    # D:/gitee
db_file_path = gitee_path / Path(r'./db')     # D:/gitee/db
db_name_stock = Path(r'./stock.db')
datapath = Path(r'./../../data/tushare_old')
dbpath = (db_file_path/db_name_stock).resolve()
ddl_path = Path(r'./sql/em_stock.sql').resolve()
print("数据库路径为 {}".format(dbpath.resolve()))

当前路径为 E:\gitee\finance\jupyterFile\prd
数据库路径为 E:\gitee\db\stock.db


In [67]:
st_date = datetime(2022,12,26)
ed_date = datetime(2022,12,31)
rd_date = datetime(2022,12,31)

st_date_ymd = st_date.strftime('%Y%m%d')
ed_date_ymd = ed_date.strftime('%Y%m%d')
rd_date_ymd = rd_date.strftime('%Y%m%d')

st_date_date = st_date.strftime('%Y-%m-%d')
ed_date_date = ed_date.strftime('%Y-%m-%d')
rd_date_date = rd_date.strftime('%Y-%m-%d')


tbl = 'em_stock_' + st_date_ymd + '_' + ed_date_ymd + '_' + rd_date_ymd
tblhfq = 'em_stock_hfq_' + st_date_ymd + '_' + ed_date_ymd + '_' + rd_date_ymd

sql = {}
sql['em_spot_data'] = """ select * from {tbl1} where `日期` <= '{end_date}' and `日期` >= '{start_date}' """.strip()
sql['em_spot_data_hfq'] = """ select * from {tblhfq1} where `日期` <= '{end_date}' and `日期` >= '{start_date}' """.strip()

# 目标路径
new_db = 'em_stock_2023_1.db'


In [68]:
with fast_db_conn(dbpath) as c:
    data = pd.read_sql(sql=sql['em_spot_data'].format(start_date = st_date_date, end_date = ed_date_date, tbl1=tbl),con = c,coerce_float = False,)
    data_hfq = pd.read_sql(sql=sql['em_spot_data_hfq'].format(start_date = st_date_date, end_date = ed_date_date, tblhfq1=tblhfq),con = c,coerce_float = False,)
    print("start_date {} end_date {} 读取成功{}".format(st_date_date,ed_date_date,data.shape))
    print("start_date {} end_date {} 读取成功{}".format(st_date_date,ed_date_date,data_hfq.shape))

# 处理数据
col_map={"日期":"d", "开盘":"open", "收盘":"close", "最高":"high", "最低":"low", "成交量":"volume", "成交额":"price_volume",
         "振幅":"amplitude","涨跌幅":"p_change","涨跌额":"price_change","换手率":"turnover",} # 股票不需要 openinterest 期货期权才涉及
h = data.rename(columns=col_map)
h1 = data_hfq.rename(columns=col_map)
h['record_date'] = rd_date_date
h1['record_date'] = rd_date_date
h = h.drop(['pk'],axis=1)
h1 = h1.drop(['pk'],axis=1)

打开连接 E:\gitee\db\stock.db
start_date 2022-12-26 end_date 2022-12-31 读取成功(25262, 13)
start_date 2022-12-26 end_date 2022-12-31 读取成功(25262, 13)
关闭连接 E:\gitee\db\stock.db


In [69]:
# 写数据
with fast_db_conn((Path(r'../db')/Path(new_db)).resolve()) as c: 
    h.to_sql(name='stock_spot', con=c, if_exists='append', index=False)
    h1.to_sql(name='stock_hfq', con=c, if_exists='append', index=False)

打开连接 E:\gitee\finance\db\em_stock_2022_4.db
关闭连接 E:\gitee\finance\db\em_stock_2022_4.db
